## CV ga oid bazi misollar

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

img = cv2.imread("/content/fil.jpg")
if img is None:
    raise ValueError("xatolik")

B, G, R = cv2.split(img)

zeros = np.zeros_like(B)

blue_img  = cv2.merge([B, zeros, zeros])
green_img = cv2.merge([zeros, G, zeros])
red_img   = cv2.merge([zeros, zeros, R])

plt.figure(figsize=(12, 6))

plt.subplot(1,4,1)
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.title("Original")
plt.axis("off")

plt.subplot(1,4,2)
plt.imshow(cv2.cvtColor(blue_img, cv2.COLOR_BGR2RGB))
plt.title("Blue Channel")
plt.axis("off")

plt.subplot(1,4,3)
plt.imshow(cv2.cvtColor(green_img, cv2.COLOR_BGR2RGB))
plt.title("Green Channel")
plt.axis("off")

plt.subplot(1,4,4)
plt.imshow(cv2.cvtColor(red_img, cv2.COLOR_BGR2RGB))
plt.title("Red Channel")
plt.axis("off")

plt.show()




In [ ]:
import cv2
from google.colab.patches import cv2_imshow

img = cv2.imread("/content/fil.jpg")

cv2_imshow(img)

print("Image shape (H, W, C):", img.shape)


In [ ]:
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
cv2_imshow(gray)
print("Gray shape:", gray.shape)


In [ ]:
resized = cv2.resize(img, (300, 100))
cv2_imshow(resized)


In [ ]:
crop = img[50:250, 50:200]  # y1:y2, x1:x2
cv2_imshow(crop)

In [ ]:
blur = cv2.GaussianBlur(img, (15, 15), 0)
cv2_imshow(blur)

In [ ]:
edges = cv2.Canny(img, 100, 200)
cv2_imshow(edges)


In [ ]:
ret, thresh = cv2.threshold(gray, 120, 255, cv2.THRESH_BINARY)
cv2_imshow(thresh)


In [ ]:
draw = img.copy()

cv2.rectangle(draw, (50, 50), (200, 200), (0, 255, 0), 3)
cv2.circle(draw, (250, 250), 50, (255, 0, 0), 3)
cv2.putText(draw, "CV!", (50, 300),
            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)

cv2_imshow(draw)


In [ ]:
h, w = img.shape[:2]
center = (w//2, h//2)
matrix = cv2.getRotationMatrix2D(center, 45, 1.0)
rotated = cv2.warpAffine(img, matrix, (w, h))
cv2_imshow(rotated)


In [ ]:
flip_h = cv2.flip(img, 1)   # horizontal
flip_v = cv2.flip(img, 0)   # vertical
cv2_imshow(flip_h)
cv2_imshow(flip_v)


# Kernel- (fiilterlar) lar

## 1. Smoothing

###  // Model structure  \\\\

##### - We define a simple CNN model with one convolution layer.
##### - A 3x3 averaging kernel(filter) is applied to produce the smoothing effect, which reduces high-frequency noise by averaging pixel values in the kernel’s neighborhood.


In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import cv2


image_path = "/content/fil.jpg"
image = Image.open(image_path)
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

input_image = transform(image).unsqueeze(0)


class SmoothingCNN(nn.Module):
    def __init__(self):
        super(SmoothingCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 1, kernel_size=3, stride=1, padding=1, bias=False)
        smoothing_kernel = np.array([[-1, 1, -1],
                                     [-1, -10, -1],
                                     [-1, 1, -1]]) / 9.0

        self.conv1.weight.data = torch.tensor(smoothing_kernel, dtype=torch.float32).repeat(1, 3, 1, 1)

    def forward(self, x):
        return self.conv1(x)
model = SmoothingCNN()
for param in model.parameters():
    param.requires_grad = False
with torch.no_grad():
    smoothed_image = model(input_image).squeeze(0).squeeze(0).numpy()
smoothed_image_uint8 = (smoothed_image * 255).astype(np.uint8)

rasm_joylashuvi = '/content/filterli_rasm.jpg'

cv2.imwrite(rasm_joylashuvi , smoothed_image_uint8)

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(np.transpose(input_image.squeeze(0).numpy(), (1, 2, 0)))
plt.title("Haqiqiy rasm")
plt.axis("off")
plt.subplot(1, 2, 2)
plt.imshow(smoothed_image, cmap="gray")
plt.title("Filterlangan rasm")
plt.axis("off")
plt.show()

print(f"Filterlangan rasmn saqlandi: {rasm_joylashuvi}")



# 2. Sharpening kernel application

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def sharpen_image(image_path, output_path):
    image = cv2.imread(image_path)

    if image is None:
        print(f"Yuklashda xatolik: {image_path}")
        return

    # O'tkirlashtiruvchi filter (kernal)
    sharpening_kernel = np.array([[0, -1, 0],
                                  [-1, 5, -1],
                                  [0, -1, 0]])

    sharpened_image = cv2.filter2D(image, -1, sharpening_kernel)
    cv2.imwrite(output_path, sharpened_image)

    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.title("Haqiqiy rasm")
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.axis("off")

    plt.subplot(1, 2, 2)
    plt.title("O'tkirlashgan rasm")
    plt.imshow(cv2.cvtColor(sharpened_image, cv2.COLOR_BGR2RGB))
    plt.axis("off")

    plt.show()

input_image_path = '/content/fil.jpg'
output_image_path = "/content/elephant_o'tkirlashgani.jpg"

sharpen_image(input_image_path, output_image_path)


# 3. Edge Detection based kernels


## 3.1. Sobel

#### Sobel operator is a discrete differentiation operator that computes an approximation of the gradient of the image intensity function


In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import cv2

def apply_kernel(input_image, kernel):
    kernel_tensor = torch.tensor(kernel, dtype=torch.float32).unsqueeze(0).unsqueeze(0)
    model = nn.Conv2d(1, 1, kernel_size=kernel_tensor.shape[2:], stride=1, padding=1, bias=False)
    model.weight.data = kernel_tensor
    output = model(input_image)
    return output.squeeze(0).squeeze(0).detach().numpy()

image_path = "/content/fil.jpg"
image = Image.open(image_path).convert("L")
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

input_image = transform(image).unsqueeze(0)

# Sobel kernel
sobel_x = np.array([[1, 0, -1],
                    [2, 0, -2],
                    [1, 0, -1]])

sobel_y = np.array([[1, 2, 1],
                    [0, 0, 0],
                    [-1, -2, -1]])

sobel_edge_x = apply_kernel(input_image, sobel_x)
sobel_edge_y = apply_kernel(input_image, sobel_y)
sobel_edges = np.sqrt(sobel_edge_x**2 + sobel_edge_y**2)

cv2.imwrite('/content/sobel_edges.jpg', (sobel_edges * 255).astype(np.uint8))

plt.figure(figsize=(10, 5))
plt.subplot(1, 3, 1)
plt.imshow(np.transpose(input_image.squeeze(0).numpy(), (1, 2, 0)), cmap='gray')
plt.title("Haqiqiy rasm")
plt.axis("off")

plt.subplot(1, 3, 2)
plt.imshow(sobel_edge_x, cmap='gray')
plt.title("Sobel Edge X")
plt.axis("off")

plt.subplot(1, 3, 3)
plt.imshow(sobel_edge_y, cmap='gray')
plt.title('Sobel Edge Y')
plt.axis("off")
plt.show()
# print("/content/sobel_edges.jpg")



# 4.2. Prewitt Edge Detection


#### Prewitt operator is similar to the Sobel operator but uses different kernels to compute gradients




In [ ]:
# Prewitt kernels
prewitt_x = np.array([[1, 0, -1],
                      [1, 0, -1],
                      [1, 0, -1]])

prewitt_y = np.array([[1, 1, 1],
                      [0, 0, 0],
                      [-1, -1, -1]])

prewitt_edge_x = apply_kernel(input_image, prewitt_x)
prewitt_edge_y = apply_kernel(input_image, prewitt_y)

prewitt_edges = np.sqrt(prewitt_edge_x**2 + prewitt_edge_y**2)

cv2.imwrite('/content/fil.jpg', (prewitt_edges * 255).astype(np.uint8))


plt.figure(figsize=(10, 5))
plt.subplot(1, 3, 1)
plt.imshow(np.transpose(input_image.squeeze(0).numpy(), (1, 2, 0)))
plt.title("Orginal rasm")
plt.axis("off")

plt.subplot(1, 3, 1)
plt.imshow(prewitt_edge_x, cmap='gray')
plt.title("Prewitt Edge X")
plt.axis("off")

plt.subplot(1, 3, 2)
plt.imshow(prewitt_edge_y, cmap='gray')
plt.title("Prewitt Edge Y ")
plt.axis("off")

plt.subplot(1, 3, 3)
plt.imshow(prewitt_edges, cmap='gray')
plt.title("Prewitt Edges")
plt.axis("off")




# **Roberts Edge Detection**

#### Roberts Cross operator uses a pair of 2x2 convolution kernels to approximate the gradient of the image

In [ ]:
# Roberts kernels
roberts_x = np.array([[1, 0],
                      [0, -1]])

roberts_y = np.array([[0, 1],
                      [-1, 0]])

roberts_edge_x = apply_kernel(input_image, roberts_x)
roberts_edge_y = apply_kernel(input_image, roberts_y)

roberts_edges = np.sqrt(roberts_edge_x**2 + roberts_edge_y**2)

cv2.imwrite('/content/elephantv2.jpg', (roberts_edges * 255).astype(np.uint8))

plt.figure(figsize=(10, 5))
plt.subplot(1, 3, 1)
plt.imshow(np.transpose(input_image.squeeze(0).numpy(), (1, 2, 0)))
plt.title("Original rasm")
plt.axis("off")

plt.subplot(1, 3, 1)
plt.imshow(roberts_edge_x, cmap='gray')
plt.title("Roberts Edge X")
plt.axis("off")

plt.subplot(1, 3, 2)
plt.imshow(roberts_edge_x, cmap='gray')
plt.title("Roberts Edge Y")
plt.axis("off")

plt.subplot(1, 3, 3)
plt.imshow(roberts_edges, cmap='gray')
plt.title("Roberts Edges")
plt.axis("off")


# **Canny Edge Detection**

#### Canny edge detection is a multi-stage algorithm that detects a wide range of edges in images

In [ ]:
# Canny Edge Detection using OpenCV
import cv2
import numpy as np
import torch
import torchvision.transforms as transforms
from PIL import Image
import matplotlib.pyplot as plt

image_path = "/content/elephantv2.jpg"
image = Image.open(image_path).convert("L")
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

input_image = transform(image).squeeze(0).numpy()

input_image_uint8 = (input_image * 255).astype(np.uint8)
edges_canny = cv2.Canny(input_image_uint8, 100, 200)

cv2.imwrite('/content/canny_edges.jpg', edges_canny)

# Display results
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(input_image, cmap='gray')
plt.title("Original rasm")
plt.axis("off")

plt.subplot(1, 2, 2)
plt.imshow(edges_canny, cmap='gray')
plt.title("Canny Edges")
plt.axis("off")




# **Summary**

---

*   **Sobel:** Measures gradients in both the x and y directions

*   **Prewitt:** Similar to Sobel but with different weights
*   **Roberts:** Focuses on a 2x2 kernel for edge detection, giving sharper results

*   **Canny:** A multi-step process that involves Gaussian smoothing and hysteresis thresholding







# Convolution with Different Kernel Sizes

### In this example, we'll create a simple grayscale image and apply convolutional filters with different kernel sizes
(ex: 3×3, 5×5, and 7×7)
### to see how each size influences the output

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt


image_path = "/content/elephantv2.jpg"
image = cv2.imread(image_path)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# kernelni (filterni aniqlash) 3x3 and 5x5 kernels
kernel_3x3 = np.array([[1, 0, -1],
                       [1, 0, -1],
                       [1, 0, -1]])

kernel_5x5 = np.array([[1, 1, 0, -1, -1],
                       [1, 1, 0, -1, -1],
                       [1, 1, 0, -1, -1],
                       [1, 1, 0, -1, -1],
                       [1, 1, 0, -1, -1]])

filtered_3x3 = cv2.filter2D(image_rgb, -1, kernel_3x3)
filtered_5x5 = cv2.filter2D(image_rgb, -1, kernel_5x5)


fig, axes = plt.subplots(1, 3, figsize=(15, 5))
axes[0].imshow(image_rgb)
axes[0].set_title("Original")

axes[1].imshow(filtered_3x3)
axes[1].set_title("Filter 3x3 Kernel)")

axes[2].imshow(filtered_5x5)
axes[2].set_title("Filter (5x5 Kernel)")

for ax in axes:
    ax.axis("off")

plt.show()


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt


image = cv2.imread('/content/a.jpg', cv2.IMREAD_GRAYSCALE)

kernels = {
    "Identity": np.array([[0, 0, 0], [0, 1, 0], [0, 0, 0]], np.float32),
    "Box Blur": np.ones((3, 3), np.float32) / 9,
    "Gaussian Blur": cv2.getGaussianKernel(3, 0) * cv2.getGaussianKernel(3, 0).T,
    "Sharpen": np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]], np.float32),
    "Edge Detection": np.array([[-1, -1, -1], [-1, 8, -1], [-1, -1, -1]], np.float32),
    "Sobel X": np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]], np.float32),
    "Sobel Y": np.array([[1, 2, 1], [0, 0, 0], [-1, -2, -1]], np.float32)
}

fig, axes = plt.subplots(2, 4, figsize=(15, 8))
axes = axes.ravel()

for i, (name, kernel) in enumerate(kernels.items()):
    filtered_image = cv2.filter2D(image, -1, kernel)
    axes[i].imshow(filtered_image, cmap='gray')
    axes[i].set_title(name)
    axes[i].axis('off')

plt.tight_layout()
plt.show()


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

image = cv2.imread('/content/a.jpg', cv2.IMREAD_GRAYSCALE)

kernels = {
    "Custom Edge Enhance": np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]], np.float32),
    "Custom Emboss": np.array([[-2, -1, 0], [-1, 1, 1], [0, 1, 2]], np.float32),
    "Large Gaussian Blur": cv2.getGaussianKernel(15, 5) * cv2.getGaussianKernel(15, 5).T,
}

def laplacian_of_gaussian(image):
    blurred = cv2.GaussianBlur(image, (5, 5), 0)
    return cv2.Laplacian(blurred, cv2.CV_64F)

def frequency_filter(image, high_pass=True, radius=30):
    dft = cv2.dft(np.float32(image), flags=cv2.DFT_COMPLEX_OUTPUT)
    dft_shift = np.fft.fftshift(dft)

    rows, cols = image.shape
    crow, ccol = rows // 2, cols // 2

    mask = np.ones((rows, cols, 2), np.uint8)
    cv2.circle(mask, (ccol, crow), radius, (0, 0), thickness=-1)
    if high_pass:
        mask = 1 - mask

    fshift = dft_shift * mask
    f_ishift = np.fft.ifftshift(fshift)
    img_back = cv2.idft(f_ishift)
    img_back = cv2.magnitude(img_back[:, :, 0], img_back[:, :, 1])
    return img_back

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.ravel()

axes[0].imshow(image, cmap='gray')
axes[0].set_title("Original Image")
axes[0].axis('off')

# Apply custom kernels
for i, (name, kernel) in enumerate(kernels.items(), 1):
    filtered_image = cv2.filter2D(image, -1, kernel)
    axes[i].imshow(filtered_image, cmap='gray')
    axes[i].set_title(name)
    axes[i].axis('off')

# Apply Laplacian of Gaussian
axes[3].imshow(laplacian_of_gaussian(image), cmap='gray')
axes[3].set_title("Laplacian of Gaussian")
axes[3].axis('off')

# Apply high-pass and low-pass frequency filters
axes[4].imshow(frequency_filter(image, high_pass=False), cmap='gray')
axes[4].set_title("Low-Pass Filter (Frequency)")
axes[4].axis('off')

axes[5].imshow(frequency_filter(image, high_pass=True), cmap='gray')
axes[5].set_title("High-Pass Filter (Frequency)")
axes[5].axis('off')

plt.tight_layout()
plt.show()


## Amaliyot


In [ ]:
import pandas as pd

animals = [
"Lion","Tiger","Elephant","Giraffe","Zebra","Cheetah","Leopard","Bear","Wolf","Fox",
"Deer","Rabbit","Kangaroo","Panda","Hippopotamus","Rhinoceros","Monkey","Gorilla","Chimpanzee",
"Hyena","Bison","Buffalo","Camel","Horse","Sheep","Goat","Dog","Cat","Moose"
]

vegetables = [
"Potato","Tomato","Cucumber","Carrot","Onion","Garlic","Pepper","Spinach","Cabbage","Broccoli",
"Cauliflower","Celery","Lettuce","Pumpkin","Radish","Beetroot","Green Bean","Pea","Eggplant",
"Zucchini","Corn","Chili","Okra","Ginger","Mushroom","Sweet Potato","Turnip","Leek","Asparagus",
"Artichoke","Parsnip","Yam","Scallion","Kale","Bok Choy","Brussels Sprout"
]

fruits = [
"Apple","Banana","Orange","Grape","Mango","Pineapple","Strawberry","Blueberry","Watermelon",
"Cherry","Peach","Pear","Plum","Kiwi","Papaya","Lemon","Lime","Apricot","Fig","Pomegranate",
"Raspberry","Blackberry","Coconut","Lychee","Guava","Mandarin","Tangerine","Dragonfruit",
"Jackfruit","Durian","Avocado","Grapefruit","Cantaloupe","Cranberry","Mulberry"
]

len(animals), len(vegetables), len(fruits)


(29, 36, 35)

In [ ]:
!pip install gradio

import gradio as gr
import pandas as pd

# Empty dataframe to store student submissions
students_df = pd.DataFrame(columns=["Name", "Student ID"])

# Function to add student entry
def add_student(name, student_id):
    global students_df

    if name.strip() == "":
        return students_df, "❌ Name cannot be empty"

    # Add to dataframe
    students_df.loc[len(students_df)] = [name, student_id]

    return students_df, f"✅ Added: {name}"

# Function to download CSV
def download_csv():
    global students_df
    file_path = "students.csv"
    students_df.to_csv(file_path, index=False)
    return file_path

# UI Layout
with gr.Blocks() as demo:
    gr.Markdown("## 📝 Student Registration Form")

    with gr.Row():
        name_input = gr.Textbox(label="Student Name")
        id_input = gr.Textbox(label="Student ID (optional)")

    submit_btn = gr.Button("Submit")
    status = gr.Textbox(label="Status")

    gr.Markdown("### 📋 Registered Students")
    table = gr.Dataframe(headers=["Name", "Student ID"], interactive=False)

    download_btn = gr.Button("Download CSV")
    file_output = gr.File(label="Download Students.csv")

    # Actions
    submit_btn.click(add_student, inputs=[name_input, id_input], outputs=[table, status])
    download_btn.click(download_csv, outputs=file_output)

demo.launch()




In [ ]:
students_df = pd.read_csv("/content/students.csv")
students_df


In [ ]:
# Combine all items
import random
all_items = animals + vegetables + fruits
random.shuffle(all_items)

# If fewer items exist, cycle them
assignments = (all_items * (len(students_df) // len(all_items) + 1))[:len(students_df)]

students_df["Assigned_Topic"] = assignments
students_df


In [ ]:
students_df.to_csv("student_topic_assignment.csv", index=False)
print("Saved as student_topic_assignment.csv")


Saved as student_topic_assignment.csv


In [ ]:
import zipfile
import os

zip_path = "/content/fil.zip"
extract_path = "/content"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Unzipped")


Unzipped


In [ ]:
import os

folder = "/content/fil"

files = sorted([f for f in os.listdir(folder) if f.lower().endswith(".jpg")])
# file larni yangi nom bilan nomlab olamiz

for i, filename in enumerate(files, start=1):
    old_path = os.path.join(folder, filename)
    new_name = f"1{i}.jpg"
    new_path = os.path.join(folder, new_name)

    os.rename(old_path, new_path)
    print(f"{filename} → {new_name}")

print("Renaming complete!")


LC202310_DavidRioOCWRangers_016_450136_reduced.jpg → 11.jpg
african-elephant-stands-on-floodplain-watching-camera (1).jpg → 12.jpg
asian-elephant.jpg → 13.jpg
cute-little-baby-elephant-mother-600nw-2467644619.jpg → 14.jpg
iStock-1458791308_523097_reduced.jpg → 15.jpg
j7u2wtpsm9yjrzumdLYC7k.jpg → 16.jpg
landscape-view-zoo-where-elephant-walking_665346-11728.jpg → 17.jpg
large-elephant-herd-walking-in-dust-in-savuti-in-botswana.jpg → 18.jpg
Renaming complete!


In [ ]:
# .jpg bo'lmagan rasmlardan qutilish, o'chirib yuborish
import os
import glob

folder_path = "/content/fil"

files = glob.glob(os.path.join(folder_path, "*"))

for file in files:
    # lowercase check to avoid ".JPG" vs ".jpg" issues
    if not file.lower().endswith(".jpg"):
        print("Deleting:", file)
        os.remove(file)

print("Done! Bizda faqat .jpg rasmlar qoldi.")



In [ ]:
import os
from PIL import Image
# malum razmerli ramlarni saqlab olamiz
folder_path = "/content/fil"
min_width, min_height = 640, 640

for filename in os.listdir(folder_path):
    if filename.lower().endswith(".jpg"):
        file_path = os.path.join(folder_path, filename)
        try:
            img = Image.open(file_path)
            w, h = img.size

            if w < min_width or h < min_height:
                print("Kichik razmerli rasmlar o'chirildi:", filename)
                os.remove(file_path)

        except:
            print(" keraksiz fayllar tashlab yuborildi", filename)


In [ ]:
import cv2
from google.colab.patches import cv2_imshow

img = cv2.imread("/content/fil/fil/12.jpg")
cv2_imshow(img)


In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import os
# umumiy rasmlarni ko'rib olamiz

folder_path = "/content/fil/fil"

for filename in os.listdir(folder_path):
    if filename.lower().endswith(".jpg"):
        img = cv2.imread(os.path.join(folder_path, filename))
        print("Bizda bor rasmlar:", filename)
        cv2_imshow(img)


In [ ]:
import os
import cv2

folder_path = "/content/fil/fil"
new_size = (224, 224)  # typical CNN input size

for filename in os.listdir(folder_path):
    if filename.lower().endswith(".jpg"):
        file_path = os.path.join(folder_path, filename)
        img = cv2.imread(file_path)
        resized = cv2.resize(img, new_size)
        cv2.imwrite(file_path, resized)


In [ ]:
import os
import cv2
from google.colab.patches import cv2_imshow
import time

folder_path = "/content/fil/fil"

for filename in os.listdir(folder_path):
    if filename.lower().endswith(".jpg"):
        img = cv2.imread(os.path.join(folder_path, filename))
        print("Displaying:", filename)
        cv2_imshow(img)
        time.sleep(1)  # show for 2 seconds


## Biz model qurish uchun kerakli rasmlarni yig'ib oldik. Endi model yaxshi ishlashligi uchun bu rasmlarni augmentatsiya qilamiz

In [ ]:
!pip install -q albumentations==1.3.1 opencv-python-headless==4.7.0.72 pillow torchvision


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.7/125.7 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 MB 19.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albucore 0.0.24 requires opencv-python-headless>=4.9.0.80, but you have opencv-python-headless 4.7.0.72 which is incompatible.


In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
# 1ta rasm misolida ko'ramiz
img_path = "/content/fil/fil/11.jpg"
img = Image.open(img_path)

plt.imshow(img)
plt.title("Original rasm")
plt.axis("off")


In [ ]:
from PIL import ImageOps
import matplotlib.pyplot as plt

#  horizontal aylantirish
flipped = ImageOps.mirror(img)

# Ko'rish
plt.figure(figsize=(10,5))

plt.subplot(1,2,1)
plt.imshow(img)
plt.title("Original")
plt.axis("off")

plt.subplot(1,2,2)
plt.imshow(flipped)
plt.title("Horizontal Flip")
plt.axis("off")


In [ ]:
from PIL import ImageOps
import matplotlib.pyplot as plt

# vertical
vflipped = ImageOps.flip(img)

plt.figure(figsize=(10,5))

plt.subplot(1,2,1)
plt.imshow(img)
plt.title("Original")
plt.axis("off")

plt.subplot(1,2,2)
plt.imshow(vflipped)
plt.title("Vertical Flip")
plt.axis("off")


In [ ]:
import random
import matplotlib.pyplot as plt

angle = random.randint(-45, 45)
rotated = img.rotate(angle)

plt.figure(figsize=(10,5))

plt.subplot(1,2,1)
plt.imshow(img)
plt.title("Original")
plt.axis("off")

plt.subplot(1,2,2)
plt.imshow(rotated)
plt.title(f"Rotation: {90}°")
plt.axis("off")


In [ ]:
import torchvision.transforms as T
import matplotlib.pyplot as plt

transform = T.RandomCrop(size=(200, 200))

cropped = transform(img)

plt.figure(figsize=(10,5))

plt.subplot(1,2,1)
plt.imshow(img)
plt.title("Original")
plt.axis("off")

plt.subplot(1,2,2)
plt.imshow(cropped)
plt.title("Random Crop")
plt.axis("off")


In [ ]:
import torchvision.transforms as T
import matplotlib.pyplot as plt

transform = T.ColorJitter(
    brightness=0.4,
    contrast=0.4,
    saturation=0.4,
    hue=0.1
)

jittered = transform(img)

plt.figure(figsize=(10,5))

plt.subplot(1,2,1)
plt.imshow(img)
plt.title("Original")
plt.axis("off")

plt.subplot(1,2,2)
plt.imshow(jittered)
plt.title("Color Jitter")
plt.axis("off")


In [ ]:
import torchvision.transforms as T
import matplotlib.pyplot as plt

transform = T.GaussianBlur(kernel_size=11)

blurred = transform(img)

plt.figure(figsize=(10,5))

plt.subplot(1,2,1)
plt.imshow(img)
plt.title("Original")
plt.axis("off")

plt.subplot(1,2,2)
plt.imshow(blurred)
plt.title("Gaussian Blur")
plt.axis("off")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

image_np = np.array(img)
noise = np.random.normal(0, 25, image_np.shape).astype('int')

noisy_image = np.clip(image_np + noise, 0, 255).astype('uint8')

plt.figure(figsize=(10,5))

plt.subplot(1,2,1)
plt.imshow(img)
plt.title("Original")
plt.axis("off")

plt.subplot(1,2,2)
plt.imshow(noisy_image)
plt.title("Random Noise")
plt.axis("off")


In [ ]:
import torchvision.transforms as T
import matplotlib.pyplot as plt

transform = T.RandomAffine(degrees=0, shear=20)

sheared = transform(img)

plt.figure(figsize=(10,5))

plt.subplot(1,2,1)
plt.imshow(img)
plt.title("Original")
plt.axis("off")

plt.subplot(1,2,2)
plt.imshow(sheared)
plt.title("Shear")
plt.axis("off")


In [ ]:
import torchvision.transforms as T
import matplotlib.pyplot as plt

transform = T.RandomPerspective(distortion_scale=0.6, p=1.0)

warped = transform(img)

plt.figure(figsize=(10,5))

plt.subplot(1,2,1)
plt.imshow(img)
plt.title("Original")
plt.axis("off")

plt.subplot(1,2,2)
plt.imshow(warped)
plt.title("Perspective Warp")
plt.axis("off")


In [ ]:
import torch
import torchvision.transforms as T
import matplotlib.pyplot as plt

transform = T.ElasticTransform(alpha=400.0, sigma=10.0)

elastic = transform(img)

plt.figure(figsize=(10,5))

plt.subplot(1,2,1)
plt.imshow(img)
plt.title("Original")
plt.axis("off")

plt.subplot(1,2,2)
plt.imshow(elastic)
plt.title("Elastic Transform")
plt.axis("off")


In [ ]:
import torchvision.transforms as T
import matplotlib.pyplot as plt

transform = T.RandomErasing(p=1.0, scale=(0.1, 0.3), ratio=(0.3, 3.0))

to_tensor = T.ToTensor()
to_pil = T.ToPILImage()

img_tensor = to_tensor(img)
cutout_tensor = transform(img_tensor)
cutout_img = to_pil(cutout_tensor)

plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
plt.imshow(img)
plt.title("Original")
plt.axis("off")

plt.subplot(1,2,2)
plt.imshow(cutout_img)
plt.title("Cutout / Random Erasing")
plt.axis("off")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

img_path2 = "/content/fil/fil/110.jpg"    # 2-rasm
img2 = Image.open(img_path2).resize(img.size)

alpha = 0.4

mixup = np.array(img) * alpha + np.array(img2) * (1 - alpha)
mixup = mixup.astype(np.uint8)

plt.figure(figsize=(15,5))

plt.subplot(1,3,1)
plt.imshow(img)
plt.title("rasm 1")
plt.axis("off")

plt.subplot(1,3,2)
plt.imshow(img2)
plt.title("rasm 2")
plt.axis("off")

plt.subplot(1,3,3)
plt.imshow(mixup)
plt.title("Mixup")
plt.axis("off")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random

img2 = Image.open("/content/fil/fil/13.jpg").resize(img.size)
img1_np = np.array(img)
img2_np = np.array(img2)

h, w, _ = img1_np.shape

# random bounding box
cut_w, cut_h = w // 3, h // 3
x = random.randint(0, w - cut_w)
y = random.randint(0, h - cut_h)

cutmix = img1_np.copy()
cutmix[y:y+cut_h, x:x+cut_w] = img2_np[y:y+cut_h, x:x+cut_w]

plt.figure(figsize=(12,5))

plt.subplot(1,2,1)
plt.imshow(img)
plt.title("Original")
plt.axis("off")

plt.subplot(1,2,2)
plt.imshow(cutmix)
plt.title("CutMix")
plt.axis("off")


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

img_cv = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2GRAY)
equalized = cv2.equalizeHist(img_cv)

plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
plt.imshow(img_cv, cmap='gray')
plt.title("Original Grayscale")
plt.axis("off")

plt.subplot(1,2,2)
plt.imshow(equalized, cmap='gray')
plt.title("Histogram Equalization")
plt.axis("off")


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

img_cv = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2GRAY)

clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
clahe_img = clahe.apply(img_cv)

plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
plt.imshow(img_cv, cmap='gray')
plt.title("Original Grayscale")
plt.axis("off")

plt.subplot(1,2,2)
plt.imshow(clahe_img, cmap='gray')
plt.title("CLAHE Result")
plt.axis("off")


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

kernel = np.array([[0, -1,  0],
                   [-1,  5, -1],
                   [0, -1,  0]])

sharpened = cv2.filter2D(np.array(img), -1, kernel)

plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
plt.imshow(img)
plt.title("Original")
plt.axis("off")

plt.subplot(1,2,2)
plt.imshow(sharpened)
plt.title("Sharpened Image")
plt.axis("off")


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

img_np = np.array(img).astype(np.float32)

h, w, _ = img_np.shape

# fog density
fog_density = np.random.uniform(0.3, 0.7)

# white fog layer
fog = np.full((h, w, 3), 255, dtype=np.float32)

# blur fog to look natural
fog = cv2.GaussianBlur(fog, (101, 101), 0)

foggy = cv2.addWeighted(img_np, 1 - fog_density, fog, fog_density, 0)
foggy = foggy.astype(np.uint8)

plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
plt.imshow(img_np.astype(np.uint8))
plt.title("Original")
plt.axis("off")

plt.subplot(1,2,2)
plt.imshow(foggy)
plt.title("Random Fog")
plt.axis("off")


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

img_np = np.array(img).astype(np.float32)

h, w, _ = img_np.shape
rain = img_np.copy()

num_drops = int((h * w) * 0.001)  # rain density
length = 20                      # drop length

for _ in range(num_drops):
    x = np.random.randint(0, w)
    y = np.random.randint(0, h)
    cv2.line(rain, (x, y), (x + np.random.randint(-2, 2), y + length), (200, 200, 200), 1)

# blur rain to look natural
rain = cv2.blur(rain, (3, 3))

plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
plt.imshow(img_np.astype(np.uint8))
plt.title("Original")
plt.axis("off")

plt.subplot(1,2,2)
plt.imshow(rain.astype(np.uint8))
plt.title("Rain Augmentation")
plt.axis("off")


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

img_np = np.array(img).astype(np.float32)

h, w, _ = img_np.shape
snow = img_np.copy()

# snow density
density = int((h * w) * 0.0015)

for _ in range(density):
    x = np.random.randint(0, w)
    y = np.random.randint(0, h)
    radius = np.random.randint(0, 3)
    cv2.circle(snow, (x, y), radius, (255, 255, 255), -1)

snow = cv2.GaussianBlur(snow, (3, 3), 0)

plt.figure(figsize=(10,5))

plt.subplot(1,2,1)
plt.imshow(img_np.astype(np.uint8))
plt.title("Original")
plt.axis("off")

plt.subplot(1,2,2)
plt.imshow(snow.astype(np.uint8))
plt.title("Snow Augmentation")
plt.axis("off")


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

img_np = np.array(img).astype(np.float32)

h, w, _ = img_np.shape

# Random shadow polygon
num_points = np.random.randint(3, 6)
points = np.array([
    [np.random.randint(0, w), np.random.randint(h//2, h)]
    for _ in range(num_points)
], dtype=np.int32)

shadow_mask = np.zeros_like(img_np, dtype=np.uint8)

# fill polygon
cv2.fillPoly(shadow_mask, [points], (255, 255, 255))

# blur shadow to make it realistic
shadow_mask = cv2.GaussianBlur(shadow_mask, (101, 101), 0)

alpha = 0.6  # shadow intensity
shadow_img = img_np * (1 - alpha * (shadow_mask / 255.0))
shadow_img = shadow_img.astype(np.uint8)

plt.figure(figsize=(10,5))

plt.subplot(1,2,1)
plt.imshow(img_np.astype(np.uint8))
plt.title("Original")
plt.axis("off")

plt.subplot(1,2,2)
plt.imshow(shadow_img)
plt.title("Random Shadow")
plt.axis("off")


In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter, map_coordinates

img_np = np.array(img).astype(np.float32)

# Elastic parameters
alpha = 40
sigma = 6

shape = img_np.shape[:2]

dx = gaussian_filter((np.random.rand(*shape) * 2 - 1), sigma) * alpha
dy = gaussian_filter((np.random.rand(*shape) * 2 - 1), sigma) * alpha

x, y = np.meshgrid(np.arange(shape[1]), np.arange(shape[0]))

coords = np.array([y + dy, x + dx])

# warp each channel
warped = np.zeros_like(img_np)
for i in range(3):
    warped[:,:,i] = map_coordinates(img_np[:,:,i], coords, order=1)

# add noise
noise = np.random.normal(0, 10, img_np.shape)
warped_noisy = np.clip(warped + noise, 0, 255).astype(np.uint8)

plt.figure(figsize=(15,5))

plt.subplot(1,3,1)
plt.imshow(img_np.astype(np.uint8))
plt.title("Original")
plt.axis("off")

plt.subplot(1,3,2)
plt.imshow(warped.astype(np.uint8))
plt.title("Elastic Warp")
plt.axis("off")

plt.subplot(1,3,3)
plt.imshow(warped_noisy)
plt.title("Elastic + Noise")
plt.axis("off")
